# Нейронные сети
- Найти данные в виде изображений для задачи классификации. Например, можно взять данные [отсюда](http://ufldl.stanford.edu/housenumbers/).
- Реализовать классы, необходимые для построения сети со следующими слоями
	- FullyConnectedLayer
	- ReluLayer
	- FullyConnectedLayer
- Использовать CrossEntropyLoss и L2-регуляризацию.
- Обучить модель на тренировочных данных, подбирать параметры (особенно learning rate) на валидационной и оценить качество на тестовой. Анализировать графики train/val loss, проверять на каждом шаге корректность вычисления градиентов с помощью разностной оценки.
- (**+2 балла**) Добавить Batch normalization.
- (**+2 балла**) В качестве оптимизатор использовать один из: Momentum, RMSprop.
- (**+1 балл**) Также реализовать оптимизатор Adam.

## Датасет
Для решения задачи был взят датасет http://ufldl.stanford.edu/housenumbers/, а именно уже разделенные тренировочные и тестовые данные, без учета дополнительных данных.

In [1]:
!pip install --quiet matplotlib
!pip install --quiet numpy
!pip install --quiet pandas 
!pip install --quiet torch
!pip install --quiet torchvision

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import SVHN
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split
from itertools import product
import pandas as pd

Подготовка датасета

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

raw_train_data = SVHN(
    root="../data/task07", 
    split='train', 
    download=True, 
    transform=transform)

test_data = SVHN(
    root="../data/task07", 
    split='test', 
    download=True, 
    transform=transform)

Создание loader-ов

In [4]:
raw_train_size = len(raw_train_data)
val_size = int(0.3 * raw_train_size)
train_size = raw_train_size - val_size

train_data, val_data = random_split(raw_train_data, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

## Слои сети